# Assignment: Data Wrangling
### `! git clone https://github.com/ds3001f25/wrangling_assignment.git`
### Do Q1 and Q2
### Reading material: `tidy_data.pdf`

**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture `./data/justice_data.parquet`, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [17]:
from google.colab import files
uploaded = files.upload()

Saving mn_police_use_of_force.csv to mn_police_use_of_force (1).csv
Saving justice_data.parquet to justice_data (1).parquet
Saving airbnb_hw.csv to airbnb_hw (1).csv


In [18]:
#question 1
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#load data
df = pd.read_csv('airbnb_hw.csv',low_memory=False)
df.head()
df.tail()

#clean up the price variable
var = "Price"
df[var] = df[var].str.replace(",", "")      # remove commas
df[var] = pd.to_numeric(df[var], errors="coerce") # convert to numbers

# check missing values
missing = df[var].isna().sum()
print(f"Missing values in {var}: {missing}")
print(df["Price"].dtype)
#end up with zero missing values


Missing values in Price: 0
int64


In [19]:
#question 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#load data
# Assuming the file is in the root of your Google Drive.
# If it's in a different folder, update the path accordingly.
df = pd.read_csv("mn_police_use_of_force.csv", low_memory=False)
df.head()
df.tail()


#Look at the raw values
var = "subject_injury"
print(df[var].unique(), '\n')   # see what categories are there

#Count each category
df[var].value_counts(dropna=False)

#Proportion missing
missing_count = 9848
total = 12925
prop_missing = missing_count / total
print(f"{prop_missing:.2%}")
# This is concerning becayse a majority of observations don't record
# whether the subject was injured. This makes the variable highly incomplete.

#cross tabulate
ct = pd.crosstab(df["subject_injury"], df["force_type"], dropna=False)
print(ct)
# A cross-tabulation of subject_injury with force_type shows that most missing
# values occur in incidents coded as Bodily Force and Chemical Irritant, while
# categories like Police K9 Bite and Taser tend to have injury outcomes more
#consistently recorded. In fact, for some force types (Maximal Restraint Technique),
# all injury outcomes are missing.
#this pattern indicates that nan likely reflects differences in reporting
#practices by force type rather than random omissions. As a result, conclusions
#drawn about injuries may underestimate injuries for certain categories of force.

[nan 'No' 'Yes'] 

76.19%
force_type      Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury                                                    
No                  0          1093                131        2   
Yes                 2          1286                 41        0   
NaN                 2          7051               1421        0   

force_type      Gun Point Display  Improvised Weapon  Less Lethal  \
subject_injury                                                      
No                             33                 34            0   
Yes                            44                 40            0   
NaN                            27                 74           87   

force_type      Less Lethal Projectile  Maximal Restraint Technique  \
subject_injury                                                        
No                                   1                            0   
Yes                                  2                            0   
NaN     

In [31]:
#question 3
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#load data
url = 'http://www.vcsc.virginia.gov/pretrialdataproject/October%202017%20Cohort_Virginia%20Pretrial%20Data%20Project_Deidentified%20FINAL%20Update_10272021.csv'
df = pd.read_csv(url,low_memory=False) # Pandas downloads and loads the .csv file for you
df.head()

# Step 1: Look at unique values
var = 'WhetherDefendantWasReleasedPretrial' # A Dummy Example

# Step 2: Replace explicit "missing" codes with NaN
df[var] = df[var].replace([9, 99, "NA", "na", "missing", "unknown", ""], np.nan)

# Step 3: Standardize Yes/No or 1/0 if present
df[var] = df[var].replace({
    "Yes": 1, "yes": 1, "1": 1,
    "No": 0, "no": 0, "0": 0
})

# Step 4: Force to numeric (allows NaN)
df[var] = pd.to_numeric(df[var], errors="coerce")

# Step 5: Check results
print(df[var].value_counts(dropna=False))
print(df[var].unique())


WhetherDefendantWasReleasedPretrial
1.0    19154
0.0     3801
NaN       31
Name: count, dtype: int64
[nan  0.  1.]


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks (Hint: `GSAF5.xls`).

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)